# nltk analyzing sentence structure

* http://www.nltk.org/book/ch08.html

* 생성문법 (Generative Grammar)

## ambiguity
* example
 * While hunting in Africa, I shot an elephant in my pajamas. How he got into my pajamas, I don't know
 

In [2]:
import nltk
grouch_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

In [3]:
sent = "I shot an elephant in my pajamas".split(" ")
parser = nltk.ChartParser(grouch_grammar)

In [5]:
for tree in parser.parse(sent):
    print tree

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


In [7]:
for tree in parser.parse(sent):
    print tree.pretty_print()

     S                                       
  ___|______________                          
 |                  VP                       
 |         _________|__________               
 |        VP                   PP            
 |    ____|___              ___|___           
 |   |        NP           |       NP        
 |   |     ___|_____       |    ___|_____     
 NP  V   Det        N      P  Det        N   
 |   |    |         |      |   |         |    
 I  shot  an     elephant  in  my     pajamas

None
     S                                   
  ___|__________                          
 |              VP                       
 |    __________|______                   
 |   |                 NP                
 |   |     ____________|___               
 |   |    |     |          PP            
 |   |    |     |       ___|___           
 |   |    |     |      |       NP        
 |   |    |     |      |    ___|_____     
 NP  V   Det    N      P  Det        N   
 |   |    |     

## 3. CFG
### 3.1 Simple Grammar

In [10]:
grammar1 = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> 'saw' | 'ate' | 'walked'
NP -> 'John' | 'Mary' | 'Bob' | Det N | Det N PP
Det -> 'a' | 'an' | 'the' | 'my' 
N -> 'man' | 'dog' | 'cat' | 'telescope' | 'park'
P -> 'in' | 'on' | 'by' | 'with'
""")

In [11]:
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)

In [13]:
for tree in rd_parser.parse(sent):
    print tree

(S (NP Mary) (VP (V saw) (NP Bob)))


In [15]:
for tree in rd_parser.parse(sent):
    tree.pretty_print()

      S         
  ____|___       
 |        VP    
 |     ___|___   
 NP   V       NP
 |    |       |  
Mary saw     Bob



* https://en.wikipedia.org/wiki/Recursive_descent_parser
 * top down parser

In [16]:
sent = "the dog saw a man in the park".split()
for tree in rd_parser.parse(sent):
    tree.pretty_print()

                 S                      
      ___________|___                    
     |               VP                 
     |        _______|___                
     |       |           NP             
     |       |    _______|___            
     |       |   |   |       PP         
     |       |   |   |    ___|___        
     NP      |   |   |   |       NP     
  ___|___    |   |   |   |    ___|___    
Det      N   V  Det  N   P  Det      N  
 |       |   |   |   |   |   |       |   
the     dog saw  a  man  in the     park

                 S                          
      ___________|_______                    
     |                   VP                 
     |        ___________|_______            
     |       |       |           PP         
     |       |       |        ___|___        
     NP      |       NP      |       NP     
  ___|___    |    ___|___    |    ___|___    
Det      N   V  Det      N   P  Det      N  
 |       |   |   |       |   |   |       |   
the  

* structurally ambiguous
 * 전치사 구의 모호성 
 * 공원에 있는 남자인지? 공원에 있는 개인지?
 * 공원에 있는 개가 남자를 봤다. 
 * 개가 공원에 있는 남자를 봤다.

### 3.2 Writing Your own grammars
* using CFG text file, we can load it into NLTK and parse

### 3.3 Recursion in Syntactic Structure

In [18]:
grammar2 = nltk.CFG.fromstring("""
  S  -> NP VP
  NP -> Det Nom | PropN
  Nom -> Adj Nom | N
  VP -> V Adj | V NP | V S | V NP PP
  PP -> P NP
  PropN -> 'Buster' | 'Chatterer' | 'Joe'
  Det -> 'the' | 'a'
  N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
  Adj  -> 'angry' | 'frightened' |  'little' | 'tall'
  V ->  'chased'  | 'saw' | 'said' | 'thought' | 'was' | 'put'
  P -> 'on'
  """)

In [19]:
rd_parser = nltk.RecursiveDescentParser(grammar2)

In [23]:
sent = "the angry bear chased the frightened little squirrel".split()
for tree in rd_parser.parse(sent):
    tree.pretty_print()

                     S                                          
       ______________|____________                               
      |                           VP                            
      |               ____________|_______                       
      |              |                    NP                    
      |              |      ______________|____                  
      NP             |     |                  Nom               
  ____|____          |     |       ____________|_____            
 |        Nom        |     |      |                 Nom         
 |     ____|___      |     |      |             _____|_____      
 |    |       Nom    |     |      |            |          Nom   
 |    |        |     |     |      |            |           |     
Det  Adj       N     V    Det    Adj          Adj          N    
 |    |        |     |     |      |            |           |     
the angry     bear chased the frightened     little     squirrel



In [24]:
sent = "Chatterer said Buster thought the tree was tall".split()
for tree in rd_parser.parse(sent):
    tree.pretty_print()

           S                                               
     ______|_____                                           
    |            VP                                        
    |       _____|_____________                             
    |      |                   S                           
    |      |      _____________|_______                     
    |      |     |                     VP                  
    |      |     |        _____________|____                
    |      |     |       |                  S              
    |      |     |       |          ________|_______        
    |      |     |       |         NP               |      
    |      |     |       |      ___|___             |       
    NP     |     NP      |     |      Nom           VP     
    |      |     |       |     |       |         ___|___    
  PropN    V   PropN     V    Det      N        V      Adj 
    |      |     |       |     |       |        |       |   
Chatterer said Buster thought th

## 4. Parsing with CFG

### 4.1 Recursive Descent Parsing
* ![image](http://www.nltk.org/images/rdparser1-6.png)